In [27]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
%cd /content/drive/MyDrive/Neo4j

/content/drive/MyDrive/Neo4j


In [29]:
!pwd

/content/drive/MyDrive/Neo4j


In [30]:
!pip install langchain langchain-community langchain-core langchain-experimental langchain-openai langchainhub langsmith py2neo neo4j openai jupyter notebook pandas

In [31]:
import os
import pandas as pd

from langchain_community.graphs import Neo4jGraph

# In local pc - to access env variables
# from dotenv import load_dotenv
# _ = load_dotenv()
from google.colab import userdata # For colab env variables

In [32]:
os.environ["NEO4J_USERNAME"] = userdata.get('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = userdata.get('NEO4J_PASSWORD')
os.environ["NEO4J_URI"] = userdata.get('NEO4J_URI')

In [33]:
df = pd.read_csv('vgsales.csv')

In [34]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [35]:
df['Rank'].nunique()==df.shape[0]

True

In [36]:
kg = Neo4jGraph()

In [39]:
vgsales_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/KNAI-AI/knai-workshop/main/data/vgsales.csv' AS row
MERGE (g:Game {rank:toInteger(row.Rank)})
SET g.name = row.Name,
    g.platform = row.Platform,
    g.year = toInteger(row.Year),
    g.genre = row.Genre,
    g.publisher = row.Publisher,
    g.na_sales = toFloat(row.NA_Sales),
    g.eu_sales = toFloat(row.EU_Sales),
    g.jp_sales = toFloat(row.JP_Sales),
    g.other_sales = toFloat(row.Other_Sales),
    g.global_sales = toFloat(row.Global_Sales)

MERGE (p:Platform {name:row.Platform})
MERGE (g)-[:RELEASED_ON]->(p)

MERGE (pub:Publisher {name:row.Publisher})
MERGE (g)-[:PUBLISHED_BY]->(pub)

MERGE (ge:Genre {name:row.Genre})
MERGE (g)-[:BELONGS_TO_GENRE]->(ge)
"""
kg.query(vgsales_query)

[]

In [40]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Game {rank: INTEGER, name: STRING, platform: STRING, year: INTEGER, genre: STRING, publisher: STRING, na_sales: FLOAT, eu_sales: FLOAT, jp_sales: FLOAT, other_sales: FLOAT, global_sales: FLOAT}
Platform {name: STRING}
Publisher {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Game)-[:RELEASED_ON]->(:Platform)
(:Game)-[:PUBLISHED_BY]->(:Publisher)
(:Game)-[:BELONGS_TO_GENRE]->(:Genre)


In [49]:
cypher = """
  MATCH (n)
  RETURN count(n) AS numberOfNodes
  """
result = kg.query(cypher)
result

[{'numberOfNodes': 17220}]

In [50]:
df['Rank'].nunique()+df['Platform'].nunique()+df['Genre'].nunique() + df['Publisher'].nunique()

17219

In [51]:
print(f"There are {result[0]['numberOfNodes']} nodes in this graph.")

There are 17220 nodes in this graph.


In [52]:
cypher = """
  MATCH (n:Game)
  RETURN count(n) AS numberOfGames
  """
kg.query(cypher)

[{'numberOfGames': 16598}]

In [54]:
cypher = """
  MATCH (nintendo:Publisher {name:"Nintendo"})
  RETURN count(nintendo) as count_nintendo
  """
kg.query(cypher)

[{'count_nintendo': 1}]

In [55]:
cypher = """
  MATCH (gta:Game {name:"Grand Theft Auto: San Andreas"})
  RETURN gta.global_sales
  """
kg.query(cypher)

[{'gta.global_sales': 20.81},
 {'gta.global_sales': 1.95},
 {'gta.global_sales': 0.98},
 {'gta.global_sales': 0.12}]

In [56]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [57]:
df[df['Name']=='Grand Theft Auto: San Andreas']

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
17,18,Grand Theft Auto: San Andreas,PS2,2004.0,Action,Take-Two Interactive,9.43,0.40,0.41,10.57,20.81
873,875,Grand Theft Auto: San Andreas,XB,2005.0,Action,Take-Two Interactive,1.26,0.61,0.00,0.09,1.95
2120,2122,Grand Theft Auto: San Andreas,PC,2005.0,Action,Take-Two Interactive,0.00,0.92,0.00,0.05,0.98
9827,9829,Grand Theft Auto: San Andreas,X360,2008.0,Action,Take-Two Interactive,0.08,0.03,0.00,0.01,0.12


In [58]:
cypher = """
  MATCH (marioBros:Game {name:"New Super Mario Bros."})
  RETURN marioBros.global_sales, marioBros.publisher
  """
kg.query(cypher)

[{'marioBros.global_sales': 30.01, 'marioBros.publisher': 'Nintendo'}]

In [59]:
cypher = """
    MATCH (g:Game)
    RETURN g.platform AS platform, SUM(g.global_sales) AS totalGlobalSales
    ORDER BY totalGlobalSales DESC
  """

kg.query(cypher)

[{'platform': 'PS2', 'totalGlobalSales': 1255.6399999999871},
 {'platform': 'X360', 'totalGlobalSales': 979.9599999999996},
 {'platform': 'PS3', 'totalGlobalSales': 957.8399999999987},
 {'platform': 'Wii', 'totalGlobalSales': 926.7099999999971},
 {'platform': 'DS', 'totalGlobalSales': 822.4899999999874},
 {'platform': 'PS', 'totalGlobalSales': 730.659999999997},
 {'platform': 'GBA', 'totalGlobalSales': 318.499999999998},
 {'platform': 'PSP', 'totalGlobalSales': 296.2799999999948},
 {'platform': 'PS4', 'totalGlobalSales': 278.0999999999994},
 {'platform': 'PC', 'totalGlobalSales': 258.81999999999846},
 {'platform': 'XB', 'totalGlobalSales': 258.25999999999834},
 {'platform': 'GB', 'totalGlobalSales': 255.44999999999987},
 {'platform': 'NES', 'totalGlobalSales': 251.06999999999988},
 {'platform': '3DS', 'totalGlobalSales': 247.4600000000001},
 {'platform': 'N64', 'totalGlobalSales': 218.87999999999985},
 {'platform': 'SNES', 'totalGlobalSales': 200.05000000000024},
 {'platform': 'GC', 't

In [62]:
cypher = """
  MATCH (nineties:Game)
  WHERE nineties.year >= 1990
    AND nineties.year < 2000
  RETURN nineties.name
  """
kg.query(cypher)

[{'nineties.name': 'Pokemon Red/Pokemon Blue'},
 {'nineties.name': 'Pokemon Gold/Pokemon Silver'},
 {'nineties.name': 'Super Mario World'},
 {'nineties.name': 'Pokémon Yellow: Special Pikachu Edition'},
 {'nineties.name': 'Super Mario 64'},
 {'nineties.name': 'Super Mario Land 2: 6 Golden Coins'},
 {'nineties.name': 'Gran Turismo'},
 {'nineties.name': 'Super Mario All-Stars'},
 {'nineties.name': 'Mario Kart 64'},
 {'nineties.name': 'Final Fantasy VII'},
 {'nineties.name': 'Gran Turismo 2'},
 {'nineties.name': 'Donkey Kong Country'},
 {'nineties.name': 'Super Mario Kart'},
 {'nineties.name': 'GoldenEye 007'},
 {'nineties.name': 'Final Fantasy VIII'},
 {'nineties.name': 'The Legend of Zelda: Ocarina of Time'},
 {'nineties.name': 'Crash Bandicoot 2: Cortex Strikes Back'},
 {'nineties.name': 'Tekken 3'},
 {'nineties.name': 'Crash Bandicoot 3: Warped'},
 {'nineties.name': 'Crash Bandicoot'},
 {'nineties.name': 'Street Fighter II: The World Warrior'},
 {'nineties.name': 'Driver'},
 {'ninetie

In [63]:
cypher = """
  MATCH (game:Game)-[:BELONGS_TO_GENRE]->(genre:Genre)
  RETURN game.name, genre.name
  """
kg.query(cypher)

[{'game.name': 'Wii Sports', 'genre.name': 'Sports'},
 {'game.name': 'Wii Sports Resort', 'genre.name': 'Sports'},
 {'game.name': 'Wii Fit', 'genre.name': 'Sports'},
 {'game.name': 'Wii Fit Plus', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 16', 'genre.name': 'Sports'},
 {'game.name': 'Mario & Sonic at the Olympic Games', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 14', 'genre.name': 'Sports'},
 {'game.name': 'Zumba Fitness', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 12', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 15', 'genre.name': 'Sports'},
 {'game.name': 'Kinect Sports', 'genre.name': 'Sports'},
 {'game.name': 'Madden NFL 2004', 'genre.name': 'Sports'},
 {'game.name': 'Mario & Sonic at the Olympic Games', 'genre.name': 'Sports'},
 {'game.name': 'FIFA Soccer 11', 'genre.name': 'Sports'},
 {'game.name': "Tony Hawk's Pro Skater", 'genre.name': 'Sports'},
 {'game.name': 'Madden NFL 06', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 15', 'genre.name': 'Sports'},
 {'game.name'

In [65]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [68]:
cypher = """
  MATCH (game:Game)
  RETURN game.name, game.genre
  """
kg.query(cypher)

[{'game.name': 'Wii Sports', 'game.genre': 'Sports'},
 {'game.name': 'Super Mario Bros.', 'game.genre': 'Platform'},
 {'game.name': 'Mario Kart Wii', 'game.genre': 'Racing'},
 {'game.name': 'Wii Sports Resort', 'game.genre': 'Sports'},
 {'game.name': 'Pokemon Red/Pokemon Blue', 'game.genre': 'Role-Playing'},
 {'game.name': 'Tetris', 'game.genre': 'Puzzle'},
 {'game.name': 'New Super Mario Bros.', 'game.genre': 'Platform'},
 {'game.name': 'Wii Play', 'game.genre': 'Misc'},
 {'game.name': 'New Super Mario Bros. Wii', 'game.genre': 'Platform'},
 {'game.name': 'Duck Hunt', 'game.genre': 'Shooter'},
 {'game.name': 'Nintendogs', 'game.genre': 'Simulation'},
 {'game.name': 'Mario Kart DS', 'game.genre': 'Racing'},
 {'game.name': 'Pokemon Gold/Pokemon Silver', 'game.genre': 'Role-Playing'},
 {'game.name': 'Wii Fit', 'game.genre': 'Sports'},
 {'game.name': 'Wii Fit Plus', 'game.genre': 'Sports'},
 {'game.name': 'Kinect Adventures!', 'game.genre': 'Misc'},
 {'game.name': 'Grand Theft Auto V', 'g

In [70]:
cypher = """
  MATCH (game:Game {name:"New Super Mario Bros."})-[:PUBLISHED_BY]->(m)<-[:PUBLISHED_BY]-(otherGames)
  RETURN otherGames.name, otherGames.global_sales
  """
kg.query(cypher)

[{'otherGames.name': 'Wii Sports', 'otherGames.global_sales': 82.74},
 {'otherGames.name': 'Super Mario Bros.', 'otherGames.global_sales': 40.24},
 {'otherGames.name': 'Mario Kart Wii', 'otherGames.global_sales': 35.82},
 {'otherGames.name': 'Wii Sports Resort', 'otherGames.global_sales': 33.0},
 {'otherGames.name': 'Pokemon Red/Pokemon Blue',
  'otherGames.global_sales': 31.37},
 {'otherGames.name': 'Tetris', 'otherGames.global_sales': 30.26},
 {'otherGames.name': 'Wii Play', 'otherGames.global_sales': 29.02},
 {'otherGames.name': 'New Super Mario Bros. Wii',
  'otherGames.global_sales': 28.62},
 {'otherGames.name': 'Duck Hunt', 'otherGames.global_sales': 28.31},
 {'otherGames.name': 'Nintendogs', 'otherGames.global_sales': 24.76},
 {'otherGames.name': 'Mario Kart DS', 'otherGames.global_sales': 23.42},
 {'otherGames.name': 'Pokemon Gold/Pokemon Silver',
  'otherGames.global_sales': 23.1},
 {'otherGames.name': 'Wii Fit', 'otherGames.global_sales': 22.72},
 {'otherGames.name': 'Wii Fit

In [72]:
cypher = """
  MATCH (game:Game {name:"New Super Mario Bros."})-[:PUBLISHED_BY]->(m)<-[:PUBLISHED_BY]-(othergame)
  RETURN othergame.name, othergame.global_sales
  """
kg.query(cypher)

[]

In [74]:
cypher = """
  MATCH (game:Game {name:"New Super Mario Bros."})-[:PUBLISHED_BY]->(m)<-[:PUBLISHED_BY]-(otherGames)
  return count(*) as rowcount
  """
kg.query(cypher)

[{'rowcount': 702}]

In [91]:
# df[df['Name']=='New Super Mario Bros.']['Publisher'].values[0]
# df[df['Publisher']==df[df['Name']=='New Super Mario Bros.']['Publisher']]
a=df[df['Publisher']==df[df['Name']=='New Super Mario Bros.']['Publisher'].values[0]]


In [93]:
a[a['Name']=='New Super Mario Bros.']

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
6,7,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.5,2.9,30.01


In [94]:
a.shape[0]

703